In [16]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [98]:
# 연속된 True 값을 계산하는 함수
def calculate_consecutive_trues(series):
    n = len(series)
    result = [0] * n
    count = 0

    # 첫 번째 패스: 연속된 True의 그룹 길이를 계산
    for i in range(n):
        if series[i]:
            count += 1
        else:
            count = 0
        result[i] = count

    # 두 번째 패스: 그룹의 마지막 True 값에 그룹의 전체 길이를 설정
    final_result = [0] * n
    i = 0
    while i < n:
        if result[i] > 0:
            length = result[i]
            for j in range(length):
                final_result[i - j] = length
            i += length
        else:
            i += 1

    return final_result

### 중국의 미세먼지 데이터를 독립변수로 사용하기 위해 처리

- 중국에서 사용할 지점은 베이징, 상하이, 난징, 광저우, 우한, 칭다오, 우루무치 7개 지점
- 각 데이터의 15년 1월 1일 ~ 24년 5월 19일까지의 데이터를 병합한 데이터 생성

In [56]:
# 중국 미세먼지 데이터 호출
beijing_dust = pd.read_csv("./중국 미세먼지 데이터/베이징-air-quality.csv")
shanghai_dust = pd.read_csv("./중국 미세먼지 데이터/상하이-air-quality.csv")
nanjing_dust = pd.read_csv("./중국 미세먼지 데이터/난징-air-quality.csv")
guangzhou_dust = pd.read_csv("./중국 미세먼지 데이터/광저우-air-quality.csv")
wuhan_dust = pd.read_csv("./중국 미세먼지 데이터/우한-air-quality.csv")
qingdao_dust = pd.read_csv("./중국 미세먼지 데이터/칭다오-air-quality.csv")
urumuchi_dust = pd.read_csv("./중국 미세먼지 데이터/우루무치-air-quality.csv")

In [58]:
# 각 지점 데이터의 date 피처 데이터 형식을 날짜로 변경
beijing_dust["date"] = beijing_dust["date"].map(lambda x: datetime.strptime(x, "%Y/%m/%d"))
shanghai_dust["date"] = shanghai_dust["date"].map(lambda x: datetime.strptime(x, "%Y/%m/%d"))
nanjing_dust["date"] = nanjing_dust["date"].map(lambda x: datetime.strptime(x, "%Y/%m/%d"))
guangzhou_dust["date"] = guangzhou_dust["date"].map(lambda x: datetime.strptime(x, "%Y/%m/%d"))
wuhan_dust["date"] = wuhan_dust["date"].map(lambda x: datetime.strptime(x, "%Y/%m/%d"))
qingdao_dust["date"] = qingdao_dust["date"].map(lambda x: datetime.strptime(x, "%Y/%m/%d"))
urumuchi_dust["date"] = urumuchi_dust["date"].map(lambda x: datetime.strptime(x, "%Y/%m/%d"))

In [59]:
# 15년도 이후의 데이터만 남기고 자르기
beijing_dust = beijing_dust[beijing_dust["date"] >= datetime.strptime("2015", "%Y")]
shanghai_dust = shanghai_dust[shanghai_dust["date"] >= datetime.strptime("2015", "%Y")]
nanjing_dust = nanjing_dust[nanjing_dust["date"] >= datetime.strptime("2015", "%Y")]
guangzhou_dust = guangzhou_dust[guangzhou_dust["date"] >= datetime.strptime("2015", "%Y")]
wuhan_dust = wuhan_dust[wuhan_dust["date"] >= datetime.strptime("2015", "%Y")]
qingdao_dust = qingdao_dust[qingdao_dust["date"] >= datetime.strptime("2015", "%Y")]
urumuchi_dust = urumuchi_dust[urumuchi_dust["date"] >= datetime.strptime("2015", "%Y")]

In [60]:
# 데이터 정렬 및 인덱스값 신규 생성
beijing_dust = beijing_dust.sort_values(by = "date").reset_index(drop = True)
shanghai_dust = shanghai_dust.sort_values(by = "date").reset_index(drop = True)
nanjing_dust = nanjing_dust.sort_values(by = "date").reset_index(drop = True)
guangzhou_dust = guangzhou_dust.sort_values(by = "date").reset_index(drop = True)
wuhan_dust = wuhan_dust.sort_values(by = "date").reset_index(drop = True)
qingdao_dust = qingdao_dust.sort_values(by = "date").reset_index(drop = True)
urumuchi_dust = urumuchi_dust.sort_values(by = "date").reset_index(drop = True)

In [61]:
# 각 지점이 어딘지 의미하는 피처 추가
beijing_dust["지점"] = "beijing"
shanghai_dust["지점"] = "shanghai"
nanjing_dust["지점"] = "nanjing"
guangzhou_dust["지점"] = "guangzhou"
wuhan_dust["지점"] = "wuhan"
qingdao_dust["지점"] = "qingdao"
urumuchi_dust["지점"] = "urumuchi"

In [81]:
# 데이터 병합
cn_dust = pd.concat([beijing_dust, shanghai_dust, nanjing_dust, guangzhou_dust, wuhan_dust, qingdao_dust, urumuchi_dust], axis = 0)

In [82]:
# ' pm25', ' pm10'에서 띄어쓰기 제거
cn_dust.columns = cn_dust.columns.str.strip(" ")

In [83]:
# 피봇 적용
cn_dust = cn_dust.pivot(index = "date", columns = "지점", values = ["pm25", "pm10"])

In [84]:
# 24년 5월 20일 이후의 데이터 제거
cn_dust = cn_dust.iloc[:-9, :]

In [85]:
# 결측된 데이터 확인
date_list = pd.date_range(start = "2015-01-01", end = "2024-05-19", freq = "1D")
missing_date = [date for date in date_list if date not in cn_dust.index]

In [86]:
# 결측된 데이터 NaN값으로 채워서 생성
for date in missing_date:
    cn_dust.loc[date, :] = np.nan

In [87]:
# date 기준으로 재정렬
cn_dust = cn_dust.sort_values(by = "date")

In [88]:
cn_dust = cn_dust.rename_axis("일시")

In [89]:
cn_dust

pm25                                                      pm10  \
지점         beijing guangzhou nanjing qingdao shanghai urumuchi wuhan beijing   
일시                                                                             
2015-01-01     124       140     133     NaN      126      NaN   156      67   
2015-01-02     114       136     160              163      NaN   178     145   
2015-01-03     210       183     182     155      173      NaN   222     141   
2015-01-04     211       162     195     201      187      NaN   224      97   
2015-01-05     128       161     234     195      158      NaN   247      45   
...            ...       ...     ...     ...      ...      ...   ...     ...   
2024-05-15      28        88      89      53       73       91   125      59   
2024-05-16      76        94      64      65       58      102   104      68   
2024-05-17     106        98      91     104       83      113   118      78   
2024-05-18     139        94      94      64       62       63   116      57   
2024-05-19     117        76      62      50       57       62   143      25   

                                                              
지점         guangzhou nanjing qingdao shanghai urumuchi wuhan  
일시                                                            
2015-01-01        81     105     NaN       73      NaN   127  
2015-01-02       108     127      69       89      NaN   181  
2015-01-03        71     139     143      100      NaN   160  
2015-01-04        66     175     121       74      NaN   153  
2015-01-05        86     124      43       78      NaN   140  
...              ...     ...     ...      ...      ...   ...  
2024-05-15                37      49       32       74    55  
2024-05-16                51      51       35      105    61  
2024-05-17                50      35       34       48    55  
2024-05-18                33      22       31       46    68  
2024-05-19                46      18       26       45    53  

[3427 rows x 14 columns]

In [105]:
na_count = [cn_dust.loc[date, :].isna().sum() for date in cn_dust.index]
na_7 = [count > 10 for count in na_count]

In [106]:
na_consec = calculate_consecutive_trues(na_count)

In [107]:
pd.Series(na_consec).value_counts()

2048    2048
0       1379
Name: count, dtype: int64